In [5]:
import geopandas as gpd
import pandas as pd
import os, sys, time
sys.path.append(r'/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets')
import GOSTnet as gn
import importlib
import networkx as nx
import osmnx as ox
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely.geometry import LineString, MultiLineString, Point

In [6]:
#read back in processed graph
G = nx.read_gpickle('./sampleData/nouakchott/biggest_subg.pickle')

In [7]:
gn.example_edge(G)

(0, 23857, {'Wkt': 'LINESTRING (-15.8975893 18.0394637, -15.8976921 18.039127)', 'id': 45978, 'infra_type': 'residential', 'osm_id': '667426151', 'key': 'edge_45978', 'length': 38.82328422750167, 'Type': 'legitimate', 'time': 13.976382321900601, 'mode': 'drive'})


In [8]:
origins = pd.read_csv('./sampleData/nouakchott/origins_test1.csv')
origins['geometry'] = list(zip(origins['Lon'],origins['Lat']))
origins['geometry'] = origins['geometry'].apply(Point)
origins_gdf = gpd.GeoDataFrame(origins, crs = {'init':'epsg:4326'}, geometry = 'geometry')
origins_gdf = gn.pandana_snap(G, origins_gdf, target_crs = 'epsg:32628', add_dist_to_node_col = True)
origins = list(origins_gdf.NN)
origins = list(set(origins))

/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets/GOSTnet.py:1623: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  G_tree = spatial.KDTree(node_gdf[['x','y']].as_matrix())
/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets/GOSTnet.py:1625: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  distances, indices = G_tree.query(in_df[['x','y']].as_matrix())


In [9]:
destinations = pd.read_csv('./sampleData/nouakchott/destinations_test1.csv')
destinations['geometry'] = list(zip(destinations['Lon'],destinations['Lat']))
destinations['geometry'] = destinations['geometry'].apply(Point)
destinations_gdf = gpd.GeoDataFrame(destinations, crs = {'init':'epsg:4326'}, geometry = 'geometry')
destinations_gdf = gn.pandana_snap(G, destinations_gdf, target_crs = 'epsg:32628', add_dist_to_node_col = True)
#destinations_gdf.NN is the nearest node of the road network
destinations = list(destinations_gdf.NN)
destinations = list(set(destinations))

In [10]:
%time OD = gn.calculate_OD(G, origins, destinations, fail_value = 9999999999999)

print origins type
<class 'list'>
weighted_origins equals false
CPU times: user 11.4 s, sys: 18 ms, total: 11.4 s
Wall time: 14.3 s


In [12]:
origins_df = pd.DataFrame(OD, columns = destinations, index = origins)

In [66]:
import importlib
importlib.reload(gn)

peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.3 
osmnx version: 0.9 


<module 'GOSTnet' from '/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets/GOSTnet.py'>

In [67]:
result2 = gn.optimize_set_coverage(origins_df)

number of origins
50
print totalCoveredFacilities
48
print percent coverage
0.96


In [68]:
result2

[15739, 1646, 2440, 25486, 33112, 4092, 4965, 6724, 7011, 8994]

In [47]:
result3 = gn.optimize_partial_set_coverage(origins_df)

In [48]:
result3

[1646, 2440]

In [49]:
result4 = gn.optimize_set_coverage(origins_df)

In [50]:
result4


[15739, 1646, 2440, 25486, 33112, 4092, 4965, 6724, 7011, 8994]